In [55]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
activity_string = 'drinking'
eating_strings = ["eating", "drinking"]
files = './WISDM/*/*/'
n = 20
subject_id = 1613

# 1. Read activity_code file

In [56]:
ss = SparkSession.builder.config('spark.driver.extraClassPath',
                                 'postgresql-42.2.18.jar')\
                         .config("spark.driver.memory", "15g")\
                         .getOrCreate()
sc = ss.sparkContext


In [57]:
endpoint = 'msds694.cmxsootjz10m.us-west-2.rds.amazonaws.com'
database = 'postgres'
table = 'activity_code'
properties = {'user': 'students', 'password': 'msdsstudents'}
url = 'jdbc:postgresql://%s/%s' % (endpoint, database)

In [58]:
activity_code = ss.read.jdbc(url=url, table=table, properties=properties)

In [59]:
activity_code.show()

+--------------------+----+
|            activity|code|
+--------------------+----+
|             Walking|   A|
|             Jogging|   B|
|              Stairs|   C|
|             Sitting|   D|
|            Standing|   E|
|              Typing|   F|
|      Brushing Teeth|   G|
|         Eating Soup|   H|
|        Eating Chips|   I|
|        Eating Pasta|   J|
|   Drinking from Cup|   K|
|     Eating Sandwich|   L|
| Kicking Soccer Ball|   M|
|Playing Catch w/T...|   O|
|Dribblinlg Basket...|   P|
|             Writing|   Q|
|            Clapping|   R|
|     Folding Clothes|   S|
+--------------------+----+



In [60]:
activity_code.select('code').distinct().count()

18

# 2. Show activity_code table

In [61]:
activity_code.select('activity', 'code').orderBy('activity', ascending=False).show()

+--------------------+----+
|            activity|code|
+--------------------+----+
|             Writing|   Q|
|             Walking|   A|
|              Typing|   F|
|            Standing|   E|
|              Stairs|   C|
|             Sitting|   D|
|Playing Catch w/T...|   O|
| Kicking Soccer Ball|   M|
|             Jogging|   B|
|     Folding Clothes|   S|
|         Eating Soup|   H|
|     Eating Sandwich|   L|
|        Eating Pasta|   J|
|        Eating Chips|   I|
|   Drinking from Cup|   K|
|Dribblinlg Basket...|   P|
|            Clapping|   R|
|      Brushing Teeth|   G|
+--------------------+----+



# 3. Reture the code with include eating string

In [65]:
def f3(x):
    determine = False
    x_lower = x.lower()
    for s in eating_strings:
        if s in x_lower:
            determine = True
    return determine
            
check_activity_w_eating = udf(f3, BooleanType())

In [66]:
activity_code.printSchema()

root
 |-- activity: string (nullable = true)
 |-- code: string (nullable = true)
 |-- eating: boolean (nullable = true)



In [67]:
activity_code = activity_code.select('activity', 'code', check_activity_w_eating('activity'))\
    .withColumnRenamed('f(activity)', 'eating')\
    .orderBy(['eating', 'code'], ascending=[False, True]) 

activity_code.show()
# activity_code.withColumn('eating', activity_code['activity'].contains(check_activity_w_eating)).show() # cannot do this

+--------------------+----+------------+
|            activity|code|f3(activity)|
+--------------------+----+------------+
|         Eating Soup|   H|        true|
|        Eating Chips|   I|        true|
|        Eating Pasta|   J|        true|
|   Drinking from Cup|   K|        true|
|     Eating Sandwich|   L|        true|
|             Walking|   A|       false|
|             Jogging|   B|       false|
|              Stairs|   C|       false|
|             Sitting|   D|       false|
|            Standing|   E|       false|
|              Typing|   F|       false|
|      Brushing Teeth|   G|       false|
| Kicking Soccer Ball|   M|       false|
|Playing Catch w/T...|   O|       false|
|Dribblinlg Basket...|   P|       false|
|             Writing|   Q|       false|
|            Clapping|   R|       false|
|     Folding Clothes|   S|       false|
+--------------------+----+------------+



# 4. Read sensor files

In [13]:
def retrive_file_name(x):
    """Returns subject_id, sensor, device and an arry of readings"""
    file_name = x[0].split("/")[-1].split(".txt")[0]
    file_arg = file_name.split("_")
    return (file_arg[1], file_arg[2], file_arg[3], x[1])


def convert_to_integer(x):
    """Convert a value to integer"""
    try:
        return int(x)
    except ValueError:
        return None


def convert_to_float(x):
    """Convert a value to float"""
    try:
        return float(x)
    except ValueError:
        return None


def conver_to_string(x):
    """Convert a value to string"""
    try:
        return str(x)
    except ValueError:
        return None


def check_same_user(x):
    """
    Return subject_id in the file name
    that is same as subject_id in the content.
    """
    if (x is not None and x[0] == x[3]):
        return (x[0], x[1], x[2], x[4], x[5], x[6], x[7], x[8])


def return_no_none_rows(x):
    """Return True if all the readings are not None"""
    if (x is not None and
            x[0] is not None and x[1] is not None and x[1] is not None and
            x[2] is not None and x[3] is not None and x[4] is not None and
            x[5] is not None and x[6] is not None and x[7] is not None):
        # if(x[5] == 0 or x[6] == 0 or x[7] == 0):
        return True
    else:
        return False


def create_flat_rdd(x):
    """
    Returns subject_id, sensor, device and
    subject_id, activity_code, x, y, z readings
    """
    values = x[3].split(",")
    if len(values) == 6:
        return (convert_to_integer(x[0]),
                x[1],
                x[2],
                convert_to_integer(values[0]),
                conver_to_string(values[1]),
                convert_to_integer(values[2]),
                convert_to_float(values[3]),
                convert_to_float(values[4]),
                convert_to_float(values[5]))


def file_rdd(ss, files):
    """Create a pair RDD using wholeTextFiles"""
    return ss.sparkContext.wholeTextFiles(files)


def create_activity_df(ss, files_rdd, schema):
    """Create dataframe using the schema"""
    activity_data_rdd = files_rdd.mapValues(lambda x: x.split(";\n"))\
        .flatMapValues(lambda x: x)\
        .map(retrive_file_name)\
        .map(create_flat_rdd)\
        .map(check_same_user)\
        .filter(return_no_none_rows)

    return ss.createDataFrame(activity_data_rdd, schema)

In [14]:
from pyspark.sql.types import *

files_rdd = file_rdd(ss, files)

schema = StructType([ StructField("subject_id", IntegerType(), False),
                      StructField("sensor", StringType(), False),
                      StructField("device", StringType(), False),
                      StructField("activity_code", StringType(), False),
                      StructField("timestamp", LongType(), False),
                      StructField("x", FloatType(), False),
                      StructField("y", FloatType(), False),
                      StructField("z", FloatType(), False)
                    ])

df_record = create_activity_df(ss, files_rdd, schema)

In [15]:
df_record.show()

+----------+------+------+-------------+---------------+------------+------------+------------+
|subject_id|sensor|device|activity_code|      timestamp|           x|           y|           z|
+----------+------+------+-------------+---------------+------------+------------+------------+
|      1613|  gyro| phone|            A|178468071944614|-0.020240024|-0.004261058|-0.023435818|
|      1613|  gyro| phone|            A|178468104194617|  -2.5750105|  0.18109496|   1.3864417|
|      1613|  gyro| phone|            A|178468142811857|  -1.5739282|   0.6668556|    1.320928|
|      1613|  gyro| phone|            A|178468183987271|  -1.5041534|   1.7973675|    0.824781|
|      1613|  gyro| phone|            A|178468225406856| -0.50786483|   1.6002935|  0.45833004|
|      1613|  gyro| phone|            A|178468263750919|   0.8072041|   1.4295849|    0.406931|
|      1613|  gyro| phone|            A|178468303909407|   2.7057717|   1.1065434|  0.22610238|
|      1613|  gyro| phone|            A|

In [16]:
df_record_distinct = df_record.select(['subject_id', 'sensor', 'device', 'activity_code']).distinct()
count_n = df_record_distinct.count()
df_record_distinct.groupBy('subject_id', 'sensor', 'device').count()\
                  .orderBy('subject_id', 'device', 'sensor').show(count_n)

+----------+------+------+-----+
|subject_id|sensor|device|count|
+----------+------+------+-----+
|      1600| accel| phone|   18|
|      1600|  gyro| phone|   18|
|      1600| accel| watch|   18|
|      1600|  gyro| watch|   18|
|      1601| accel| phone|   18|
|      1601|  gyro| phone|   18|
|      1601| accel| watch|   18|
|      1601|  gyro| watch|   18|
|      1602| accel| phone|   18|
|      1602|  gyro| phone|   18|
|      1602| accel| watch|   18|
|      1602|  gyro| watch|   18|
|      1603| accel| phone|   18|
|      1603|  gyro| phone|   18|
|      1603| accel| watch|   18|
|      1603|  gyro| watch|   18|
|      1604| accel| phone|   18|
|      1604|  gyro| phone|   18|
|      1604| accel| watch|   18|
|      1604|  gyro| watch|   18|
|      1605| accel| phone|   18|
|      1605|  gyro| phone|   18|
|      1605| accel| watch|   18|
|      1605|  gyro| watch|   18|
|      1606| accel| phone|   18|
|      1606|  gyro| phone|   18|
|      1606| accel| watch|   18|
|      160

# 5. Summary statistics for coordinates

In [17]:

cols_groupBy_5 = ['subject_id', 'activity', 'device', 'sensor']
cols_orderBy_5 = ['activity', 'subject_id', 'device', 'sensor']


In [18]:
df_record_activity = df_record.join(activity_code, df_record.activity_code == activity_code.code)
df_record_activity = df_record_activity.repartition('activity', 'subject_id', 'device', 'sensor').cache()
df_record_activity.show()

+----------+------+------+-------------+--------------+-------------+-------------+------------+--------+----+------+
|subject_id|sensor|device|activity_code|     timestamp|            x|            y|           z|activity|code|eating|
+----------+------+------+-------------+--------------+-------------+-------------+------------+--------+----+------+
|      1608|  gyro| phone|            F|44355302955337|   0.29908752|   0.26802063|   0.5319214|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355353309341|   0.14956665|   0.08605957| 0.117767334|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355403663345|  0.033935547| -0.021224976| 0.020996094|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355454017349|  0.009490967|-0.0048675537| 0.011276245|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355504371353| 0.0048980713|  0.039123535|-0.006072998|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355554725357| 

In [19]:
df_record_activity.groupBy(cols_groupBy_5).agg(min('x').alias('x_min')\
                                     , min('y').alias('y_min')\
                                     , min('z').alias('z_min')\
                                     , avg('x').alias('x_avg')\
                                     , avg('y').alias('y_avg')\
                                     , avg('z').alias('z_avg')\
                                     , max('x').alias('x_max')\
                                     , max('y').alias('y_max')\
                                     , max('z').alias('z_max')\
                                     , expr('percentile(x, 0.05)').alias('x_05%')\
                                     , expr('percentile(y, 0.05)').alias('y_05%')\
                                     , expr('percentile(z, 0.05)').alias('z_05%')\
                                     , expr('percentile(x, 0.25)').alias('x_25%')\
                                     , expr('percentile(y, 0.25)').alias('y_25%')\
                                     , expr('percentile(z, 0.25)').alias('z_25%')\
                                     , expr('percentile(x, 0.5)').alias('x_50%')\
                                     , expr('percentile(y, 0.5)').alias('y_50%')\
                                     , expr('percentile(z, 0.5)').alias('z_50%')\
                                     , expr('percentile(x, 0.75)').alias('x_75%')\
                                     , expr('percentile(y, 0.75)').alias('y_75%')\
                                     , expr('percentile(z, 0.75)').alias('z_75%')\
                                     , expr('percentile(x, 0.95)').alias('x_95%')\
                                     , expr('percentile(y, 0.95)').alias('x_95%')\
                                     , expr('percentile(z, 0.95)').alias('x_95%')\
                                     , stddev('x').alias('x_std')\
                                     , stddev('y').alias('y_std')\
                                     , stddev('z').alias('z_std')).orderBy(cols_orderBy_5).show(n)


+----------+--------------+------+------+-----------+-----------+-----------+--------------------+--------------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|subject_id|      activity|device|sensor|      x_min|      y_min|      z_min|               x_avg|               y_avg|               z_avg|      x_max|     y_max|     z_max|               x_05%|               y_05%|               z_05%|               x_25%|               y_25%|               z_25%|               x_50%|               y_50%|               z_50%|               x_75%|               y_75%|               z_75%|               x_95%| 

# 6. Show the records with activity_string

In [20]:
def f(x, y):
    x_lower = x.lower()
    if activity_string in x_lower and y == subject_id:
        return True
    else:
        return False
            
check_activity_w_activity = udf(f, BooleanType())

In [21]:
cols_select_6 = ['activity', 'timestamp', 'device', 'sensor', 'x', 'y', 'z' ]
cols_orderBy_6 = ['timestamp', 'device', 'sensor']

In [22]:
df_record_activity.filter(check_activity_w_activity('activity', 'subject_id'))\
                .select(cols_select_6)\
                .orderBy(cols_orderBy_6).show(n)

# cols_orderBy_6, ascending=[True, False, False]

+-----------------+---------------+------+------+------------+------------+------------+
|         activity|      timestamp|device|sensor|           x|           y|           z|
+-----------------+---------------+------+------+------------+------------+------------+
|Drinking from Cup|175766024086015| phone| accel|   1.0630256|   5.0996494|    8.136096|
|Drinking from Cup|175766024086015| phone|  gyro|-0.020240024|-0.004261058|-0.023435818|
|Drinking from Cup|175766059746973| phone| accel|   1.4281414|    5.017648|    7.902063|
|Drinking from Cup|175766059746973| phone|  gyro| -0.06844324| -0.14940333|-0.056459017|
|Drinking from Cup|175766099610941| phone| accel|   1.6065093|   5.3881507|    7.901464|
|Drinking from Cup|175766099610941| phone|  gyro|-0.049534798|-0.083889574|-0.059921127|
|Drinking from Cup|175766139549562| phone| accel|   1.2575544|   5.4102974|   7.9906483|
|Drinking from Cup|175766139549562| phone|  gyro| -0.07829694| -0.01917476| -0.03195793|
|Drinking from Cup|17

# 7. Create features for the coordinates for accelerometer and gyroscope

In [23]:
def f7(x, y, z):
    """
    x: activity
    y: subject_id
    z:count
    """
    x_lower = x.lower()
    if activity_string in x_lower and y == subject_id and z >= 2:
        return True
    else:
        return False
            
check_activity_id_count = udf(f7, BooleanType())

In [24]:
cols_partitionBy_7 = ['activity_code', 'device', 'timestamp']
cols_select_7 = ['activity_code', 'device', 'timestamp', 'x', 'y', 'z']
windowsSpec = Window.partitionBy(cols_partitionBy_7)

In [29]:
df_record_activity.show()

+----------+------+------+-------------+--------------+-------------+-------------+------------+--------+----+------+
|subject_id|sensor|device|activity_code|     timestamp|            x|            y|           z|activity|code|eating|
+----------+------+------+-------------+--------------+-------------+-------------+------------+--------+----+------+
|      1608|  gyro| phone|            F|44355302955337|   0.29908752|   0.26802063|   0.5319214|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355353309341|   0.14956665|   0.08605957| 0.117767334|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355403663345|  0.033935547| -0.021224976| 0.020996094|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355454017349|  0.009490967|-0.0048675537| 0.011276245|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355504371353| 0.0048980713|  0.039123535|-0.006072998|  Typing|   F| false|
|      1608|  gyro| phone|            F|44355554725357| 

In [48]:
df_accel = df_record_activity.select(*df_record_activity.columns, count('sensor').over(windowsSpec).alias('count'))\
                              .filter(check_activity_id_count('activity', 'subject_id', 'count'))\
                              .filter("sensor == 'accel'")\
                              .select(cols_select_7)\
                              .withColumnRenamed('x', 'accel_x')\
                              .withColumnRenamed('y', 'accel_y')\
                              .withColumnRenamed('z', 'accel_z')

df_gyro = df_record_activity.select(*df_record_activity.columns, count('sensor').over(windowsSpec).alias('count'))\
                              .filter(check_activity_id_count('activity', 'subject_id', 'count'))\
                              .filter("sensor == 'gyro'")\
                              .select(cols_select_7)\
                              .withColumnRenamed('x', 'gyro_x')\
                              .withColumnRenamed('y', 'gyro_y')\
                              .withColumnRenamed('z', 'gyro_z')

In [49]:
cols_rearrange_7= ['activity_code', 'device', 'timestamp', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

cond = (df_accel.activity_code == df_gyro.activity_code)\
        & (df_accel.device == df_gyro.device)\
        & (df_accel.timestamp == df_gyro.timestamp)

df_accel.join(df_gyro, cond, 'left')\
        .drop(df_gyro.device)\
        .drop(df_gyro.activity_code)\
        .drop(df_gyro.timestamp)\
        .select(cols_rearrange_7)\
        .orderBy('timestamp')\
        .show(n)


+-------------+------+---------------+----------+---------+---------+-------------+------------+------------+
|activity_code|device|      timestamp|   accel_x|  accel_y|  accel_z|       gyro_x|      gyro_y|      gyro_z|
+-------------+------+---------------+----------+---------+---------+-------------+------------+------------+
|            K| phone|175766024086015| 1.0630256|5.0996494| 8.136096| -0.020240024|-0.004261058|-0.023435818|
|            K| phone|175766059746973| 1.4281414| 5.017648| 7.902063|  -0.06844324| -0.14940333|-0.056459017|
|            K| phone|175766099610941| 1.6065093|5.3881507| 7.901464| -0.049534798|-0.083889574|-0.059921127|
|            K| phone|175766139549562| 1.2575544|5.4102974|7.9906483|  -0.07829694| -0.01917476| -0.03195793|
|            K| phone|175766179413402| 1.0696096|5.1050367| 8.134899| -0.061519023|  0.01438107| 0.007989483|
|            K| phone|175766219317073| 1.2886791|5.1170077| 8.032547| -0.051931642| 0.015978966| 0.002396845|
|         

In [50]:
ss.stop()